In [ ]:
!pip install cdsapi numpy urllib3  pandas datetime xarray[complete] 

In [4]:
import xarray as xr
# plt.style.available
#import colormaps as cmaps 
#import concurrent
import numpy as np
# # Libraries to assist with animation and visualisations
# %matplotlib inline
# import matplotlib.pyplot as plt
# from matplotlib import animation
from IPython.display import HTML

# Disable warnings for data download via API
import urllib3
urllib3.disable_warnings()

import json
import pandas as pd
from datetime import datetime, timedelta
import xarray as xr
# import cfgrib
# import eccodes
# import geopandas as gpd
# from shapely.geometry import Point
import zipfile
import os
import logging

from zipfile import ZipFile 
import cdsapi
URL = 'https://ads.atmosphere.copernicus.eu/api'
KEY = '87bb8d36-2458-48c2-b5e3-f1fd352291bc'


In [2]:

logging.basicConfig(level=logging.INFO)

def dict_l(data, fm):
    try:
        # Validate filename
       
        
        # Convert data to dictionary
        data_dict = data.squeeze().to_dict(data='list', encoding=False)
        
        # Helper function to recursively round numeric values
        def round_numbers(obj):
            if isinstance(obj, dict):
                return {k: round_numbers(v) for k, v in obj.items()}
            elif isinstance(obj, list):
                return [round_numbers(item) for item in obj]
            elif isinstance(obj, (float, np.float32, np.float64)):
                return round(float(obj), 2)  # Round floats to 2 decimal places
            elif isinstance(obj, (int, np.int32, np.int64)):
                return int(obj)  # Keep integers as-is
            else:
                return obj  # Leave other types unchanged
        
        # Apply rounding to the dictionary
        rounded_data_dict = round_numbers(data_dict)
        
        # Helper function to handle non-serializable types
        def convert_types(obj):
            if isinstance(obj, dict):
                return {k: convert_types(v) for k, v in obj.items()}
            elif isinstance(obj, list):
                return [convert_types(item) for item in obj]
            elif isinstance(obj, (np.datetime64, pd.Timestamp, datetime)):
                return obj.isoformat()
            elif isinstance(obj, timedelta):
                return obj.total_seconds()
            elif isinstance(obj, np.ndarray):
                return obj.tolist()
            elif isinstance(obj, (np.float32, np.float64)):
                return float(obj)
            elif isinstance(obj, (np.int32, np.int64)):
                return int(obj)
            raise TypeError(f"Type {type(obj)} not serializable")
        
        # Save to JSON file
        file = os.path.join(os.getcwd(), f"{fm}.json")
        with open(file, 'w') as json_file:
            json.dump(rounded_data_dict, json_file, indent=4, default=convert_types)
            
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        raise

In [6]:
from datetime import date,timedelta
time1=(date.today()- timedelta(days = 1)).strftime("%Y-%m-%d")
time1


'2025-03-27'

In [ ]:

dataset = "cams-global-atmospheric-composition-forecasts"
lead= [str(i) for i in range(0,73)]

request = {
    "date": [f"{time1}/{time1}"],
    "time": ["12:00"],
    "leadtime_hour": lead,
    "type": ["forecast"],
    "data_format": "netcdf_zip",
    "variable": [
         "2m_dewpoint_temperature",
        "dust_aerosol_optical_depth_550nm",
        "mean_sea_level_pressure",
        "nitrate_aerosol_optical_depth_550nm",
        "particulate_matter_2.5um",
        "particulate_matter_10um",
        "sea_salt_aerosol_optical_depth_550nm",
        "sulphate_aerosol_optical_depth_550nm",
        "total_aerosol_optical_depth_550nm",
        "convective_precipitation",
        "high_cloud_cover",
        "low_cloud_cover",
        "medium_cloud_cover",
        "total_cloud_cover",
        "most_unstable_convective_available_potential_energy",
        "most_unstable_convective_inhibition"

    ],
    "area": [50, 50, 0, 100]
}

client = cdsapi.Client(url=URL, key=KEY)
client.retrieve(dataset, request).download('test.zip')


In [ ]:

with ZipFile("test.zip", 'r') as zObject: 
    zObject.extractall(path="./extracted_files")  # Specify a different path for extraction

ds1 = xr.open_dataset('./extracted_files/data_sfc.nc')  # Update the path to the extracted file
os.rm('test.zip')

In [ ]:
ds=ds1.copy(deep=True)
# Given information
forecast_ref_time = ds['forecast_reference_time'].values
hours=len(ds['forecast_period'])  # Number of forecast periods

if isinstance(forecast_ref_time, (np.ndarray, xr.DataArray)):
    forecast_ref_time = forecast_ref_time.item()  # or forecast_ref_time[0]

# Convert to datetime if it's numpy datetime64
forecast_ref_time = pd.to_datetime(forecast_ref_time)



valid_time = pd.date_range(start=forecast_ref_time,  periods=hours,  # 100 hours = 100 steps
    freq='1h'     # '1H' = 1-hour frequency
)

# Verify the length matches your existing time dimension
print(f"Original time length: {len(ds['forecast_period'])}")
print(f"New time1 length: {len(valid_time)}")

# Assign the new coordinate to your dataset
ds = ds.assign_coords(valid_time=('forecast_period', valid_time))

# Now you can filter for JJAS months using the new time coordinate

# If you want to make time1 the main dimension coordinate:
ds = ds.swap_dims({'forecast_period': 'valid_time'})
ds = ds.drop_vars('forecast_period')  # optional - remove the old time coordinate


Original time length: 73
New time1 length: 73


In [14]:
aod_du=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['duaod550']
aod_ss=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['ssaod550']
aod_su=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['suaod550']
aod=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['aod550']
aod_ni=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['niaod550']
time=ds.valid_time






pm25=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['pm2p5']
pm10=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['pm10']

lcc=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['lcc']
mcc=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['mcc']
hcc=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['hcc']
tcc=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['tcc']

cape=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['mucape']
cin=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['mucin']

d2m=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['d2m']
cp=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['cp']



In [15]:
 dict_l(1e9*pm25,"PM25")
 dict_l(1e9*pm10,"PM10")

 dict_l(lcc,"LCC")
 dict_l(mcc,"MCC")
 dict_l(hcc,"HCC")
 dict_l(tcc,"TCC")

 dict_l(cape,"CAPE")
 dict_l(cin,"CIN")
 dict_l(d2m-273.15,"D2M")
 dict_l(cp*1000,"CP")

 dict_l(aod_ss,"AOD_SEA")
 dict_l(aod_su,"AOD_SUL")
 dict_l(aod_ni,"AOD_NIT")
 dict_l(aod_du,"AOD_DUST")
 dict_l(aod,"AOD_TOT")

dict_l(time,"TIME")






In [ ]:
# import numpy as np

# # Define the colors in RGBA format
# colors = {
#     "blue": (0, 0, 255, 1),
#     "aqua": (0, 255, 255, 1),
#     "green": (0, 255, 0, 1),
#     "yellow": (255, 255, 0, 1),
#     "orange": (255, 165, 0, 1),
#     "red": (255, 0, 0, 1)
# }

# # Define color stops and corresponding positions
# color_stops = [
#     (0.0, (0, 0, 0, 1)),  # Changed to tuple
#     (0.12, (0, 0, 255, 1)),  # Changed to tuple
#     (0.25, (0, 255, 255, 1)),  # Changed to tuple
#     (0.38, (0, 255, 0, 1)),  # Changed to tuple
#     (0.5, (255, 255, 0, 1)),  # Changed to tuple
#     (0.62, (255, 165, 0, 1)),  # Changed to tuple
#     (0.75, (255, 0, 0, 1)),  # Changed to tuple
#     (0.88, (238, 130, 238, 1)),  # Changed to tuple
#     (1.0, (255, 255, 255, 1))  # Changed to tuple
# ]

# # Generate 100 interpolated values
# n_points = 200
# interpolated_colors = []

# for i in range(n_points):
#     pos = i / (n_points - 1)
#     # Find the two closest stops
#     for j in range(len(color_stops) - 1):
#         if color_stops[j][0] <= pos <= color_stops[j + 1][0]:
#             # Interpolate between the two stops
#             start_pos, start_color = color_stops[j]
#             end_pos, end_color = color_stops[j + 1]
#             ratio = (pos - start_pos) / (end_pos - start_pos)
#             interpolated_color = tuple(
#                 int(start_color[k] + ratio * (end_color[k] - start_color[k])) if k < 3 else start_color[k]
#                 for k in range(4)
#             )
#             rgba_string = f"rgba({interpolated_color[0]}, {interpolated_color[1]}, {interpolated_color[2]}, {interpolated_color[3]})"
#             interpolated_colors.append([round(pos, 2), rgba_string])
#             break

# interpolated_colors


In [ ]:
# # Number of interpolated colors
# n_points = 256

# # Prepare an empty list for the AMWG256 colormap
# amwg256 = []

# # Define the positions of the input colors in the sequence
# color_positions = [i / (len(color_sequence) - 1) for i in range(len(color_sequence))]

# # Generate interpolated colors for 256 points
# for i in range(n_points):
#     pos = i / (n_points - 1)
#     # Find the two closest positions in the sequence
#     for j in range(len(color_positions) - 1):
#         if color_positions[j] <= pos <= color_positions[j + 1]:
#             # Interpolate between the two colors
#             start_pos, end_pos = color_positions[j], color_positions[j + 1]
#             start_color, end_color = color_map[color_sequence[j]], color_map[color_sequence[j + 1]]
#             ratio = (pos - start_pos) / (end_pos - start_pos)
#             interpolated_color = tuple(
#                 int(start_color[k] + ratio * (end_color[k] - start_color[k])) if k < 3 else start_color[k]
#                 for k in range(4)
#             )
#             rgba_string = f"rgba({interpolated_color[0]}, {interpolated_color[1]}, {interpolated_color[2]}, {interpolated_color[3]})"
#             amwg256.append([round(pos, 3), rgba_string])
#             break

# amwg256  # Display the first 10 color stops to verify


In [ ]:
# # Define the colormap structure for GMT_panoply with stops: Black, Blue, Aqua, Green, Yellow, Orange, Red, White
# gmt_color_sequence = ["Bk", "Bl", "Aq", "Gr", "Ye", "Or", "Re", "Wh"]

# # Define the positions of the input colors
# gmt_positions = [i / (len(gmt_color_sequence) - 1) for i in range(len(gmt_color_sequence))]

# # Interpolating 256 colors for the GMT_panoply colormap
# gmt_panoply_256 = []

# for i in range(n_points):
#     pos = i / (n_points - 1)
#     # Find the two closest positions in the sequence
#     for j in range(len(gmt_positions) - 1):
#         if gmt_positions[j] <= pos <= gmt_positions[j + 1]:
#             # Interpolate between the two colors
#             start_pos, end_pos = gmt_positions[j], gmt_positions[j + 1]
#             start_color, end_color = color_map[gmt_color_sequence[j]], color_map[gmt_color_sequence[j + 1]]
#             ratio = (pos - start_pos) / (end_pos - start_pos)
#             interpolated_color = tuple(
#                 int(start_color[k] + ratio * (end_color[k] - start_color[k])) if k < 3 else start_color[k]
#                 for k in range(4)
#             )
#             rgba_string = f"rgba({interpolated_color[0]}, {interpolated_color[1]}, {interpolated_color[2]}, {interpolated_color[3]})"
#             gmt_panoply_256.append([round(pos, 3), rgba_string])
#             break

# gmt_panoply_256  # Display the first 10 color stops to verify
